In [1]:
# If you have your csv in "data" folder in your current directory and sub-folders for the values you want to iterate
# you can specify the name of the columns to plot your graphs

# Cleaner explanation will be added

In [39]:
import re
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
from functools import partial

In [ ]:
regex = "(\d+\.\d+)\s+(\w+)"

In [40]:
def convert_to_snakecase(regex, s):
    x = re.search(regex, s)
    return x.group(1) + "_" + x.group(2) 
    

In [41]:
def create_directory(path, parameter, model):
    try:
        os.makedirs(path/parameter/model)
    except OSError as e:
        pass
    

In [42]:
def plot_manager(x, y, x_label="", y_label="", title="Pressure", path = Path("./data"), parameter_name_converter = None):
    for p,d,f in os.walk(path):
        pth = Path(p)
        if pth.name == "models":
                d[:] = []
                models = list(pth.iterdir())
                parameter = parameter_name_converter(pth.parent.name)
                model_df = {}
                

                for fl in os.listdir(models[0]):
                    cross_section = fl[0]
                    iter_models = iter(models)
                    first_m = next(iter_models)
                    df = pd.read_csv(models[0]/fl)
                    model_df[models[0].name] = df


                    for m in iter_models:
                        for fl2 in os.listdir(m):
                            if fl2.startswith(cross_section):
                                model_df[m.name] = pd.read_csv(m/fl2)
                                break

                    
                    
                    combine_plot(model_df, parameter, cross_section, x, y, x_label, y_label, title, dpi = 300)






In [43]:
def plot(df, x, y, title, x_label, y_label, cross_section, parameter, model, dpi):
    plt.plot(df[x],df[y])
    plt.title(title + " at " + str(cross_section))
    plt.xlabel(x if x_label is None else x_label)
    plt.ylabel(y if y_label is None else y_label)
    create_directory(Path("."/Path("plots")/Path(str(title))), parameter, model)
    dir_path = Path("."/Path("plots")/Path(str(title)))/parameter/model
    plt.savefig(dir_path/str(title + "_" + str(cross_section) + "_" + parameter + "_" + model + ".jpg"), dpi = dpi)
    plt.clf()
        

In [44]:
def same_graph(x, y, title, x_label, y_label, cross_section, parameter, model_df, dpi):
    
    for model, df in model_df.items():
        plt.plot(df[x],df[y], label = model)
    plt.title(title + " at " + str(cross_section))
    plt.xlabel(x if x_label is None else x_label)
    plt.ylabel(y if y_label is None else y_label)
    model_combined = "_".join(list(model_df.keys())).upper()
    create_directory(Path("."/Path("plots")/Path(str(title))), parameter, model_combined)
    dir_path = Path("."/Path("plots")/Path(str(title)))/parameter/model_combined
    plt.savefig(dir_path/str(title + "_" + str(cross_section) + "_" + parameter + "_" + model_combined + ".jpg"), dpi = dpi)
    plt.clf()
    

In [50]:
# model_df is a dictionary containing model and df pair
def combine_plot(model_df, parameter, cross_section, x, y, x_label, y_label, title, dpi = 300):
    if len(model_df) == 1:
        model = list(model_df.keys())[0]
        df = list(model_df.values())[0]
        plot(df, x, y, title, x_label, y_label, cross_section, parameter, model, dpi)
    
    else:
        for k in model_df:
            model = k
            df = model_df[k]
            plot(df, x, y, title, x_label, y_label, cross_section, parameter, model, dpi)
        
        
        same_graph(x, y, title, x_label, y_label, cross_section, parameter, model_df, dpi)
        
        
        

In [ ]:
parameter_name_converter = partial(convert_to_snakecase, regex)

In [49]:
plot_manager("arc_length","p", title="Pressure", parameter_name_converter = parameter_name_converter)

<Figure size 432x288 with 0 Axes>

In [47]:
plot_manager("arc_length","U:1", title="Velocity", parameter_name_converter = parameter_name_converter)

<Figure size 432x288 with 0 Axes>